<a href="https://colab.research.google.com/github/m-bashari-m/vehicle-color-recognition/blob/main/src/02_rgb_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Prepare utils file

!wget https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py

--2022-06-16 23:13:28--  https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11745 (11K) [text/plain]
Saving to: ‘utils.py.1’

utils.py.1          100%[===================>]  11.47K  --.-KB/s    in 0s      

2022-06-16 23:13:28 (121 MB/s) - ‘utils.py.1’ saved [11745/11745]



In [2]:
from google.colab import drive
drive.mount('./drive')

Drive already mounted at ./drive; to attempt to forcibly remount, call drive.mount("./drive", force_remount=True).


In [4]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

from utils import ModelCreator, get_train_val_ds, ErrorAnalyzer, get_class_weight

In [5]:
BATCH_SIZE = 32
IMG_SIZE = (256, 256)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16
HUB_URL = 'https://tfhub.dev/google/bit/m-r50x1/1'

In [6]:
dataset_dir = os.path.join('drive', 'MyDrive', 'cars')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [7]:
# Get shuffled train and validation dataset with one-hot labels

train_ds, val_ds = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE, img_size=IMG_SIZE)

Found 17043 files belonging to 16 classes.
Found 3495 files belonging to 16 classes.


In [8]:
# Get class weights which have calculated from the previous notebook and stored in github

classes, class_weight = get_class_weight()

In [9]:
for color, weight in zip(classes, class_weight.values()):
  print(color, "    \t=>", weight)

beige     	=> 17.94
black     	=> 13.8
blue     	=> 15.47
brown     	=> 17.92
cream     	=> 56.81
crimson     	=> 76.43
gold     	=> 56.81
green     	=> 21.2
grey     	=> 12.41
navy-blue     	=> 58.77
orange     	=> 22.37
red     	=> 16.39
silver     	=> 20.17
titanium     	=> 56.81
white     	=> 2.97
yellow     	=> 20.68


In [10]:
train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [11]:
rgb_creator = ModelCreator(HUB_URL, 'RGB-model')
rgb_model = rgb_creator.make_model(img_size=IMG_SIZE)

Model: "RGB-model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 2048)              23500352  
                                                                 
 dense (Dense)               (None, 16)                32784     
                                                                 
Total params: 23,533,136
Trainable params: 32,784
Non-trainable params: 23,500,352
_________________________________________________________________


In [12]:
# Callbacks are EarlyStopping and ModelCheckpoint

callbacks = rgb_creator.get_callbacks()

In [13]:
history = rgb_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=20,
                        class_weight=class_weight)

Epoch 1/20
533/533 [==============================] - 512s 931ms/step - loss: 114.3607 - auc: 0.5134 - accuracy: 0.6061
Epoch 2/20
533/533 [==============================] - 148s 275ms/step - loss: 75.4070 - auc: 0.6443 - accuracy: 0.7261
Epoch 3/20
533/533 [==============================] - 150s 280ms/step - loss: 63.6368 - auc: 0.7018 - accuracy: 0.7717
Epoch 4/20
533/533 [==============================] - 150s 279ms/step - loss: 54.8657 - auc: 0.7316 - accuracy: 0.7946
Epoch 5/20
533/533 [==============================] - 150s 279ms/step - loss: 47.9548 - auc: 0.7540 - accuracy: 0.8094
Epoch 6/20
533/533 [==============================] - 150s 280ms/step - loss: 40.7218 - auc: 0.7723 - accuracy: 0.8246
Epoch 7/20
533/533 [==============================] - 150s 280ms/step - loss: 29.8010 - auc: 0.8183 - accuracy: 0.8530
Epoch 8/20
533/533 [==============================] - 150s 280ms/step - loss: 26.2726 - auc: 0.8292 - accuracy: 0.8601
Epoch 9/20
533/533 [===========================

In [ ]:
history = rgb_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=25,
                        initial_epoch=10,
                        class_weight=class_weight)

Epoch 11/25
533/533 [==============================] - 154s 286ms/step - loss: 14.8086 - auc: 0.8846 - accuracy: 0.8963
Epoch 12/25
533/533 [==============================] - 153s 286ms/step - loss: 15.4249 - auc: 0.8774 - accuracy: 0.8910
Epoch 13/25
533/533 [==============================] - 153s 286ms/step - loss: 11.6201 - auc: 0.9081 - accuracy: 0.9116
Epoch 14/25
533/533 [==============================] - 154s 286ms/step - loss: 9.5923 - auc: 0.9162 - accuracy: 0.9149
Epoch 15/25
533/533 [==============================] - 153s 285ms/step - loss: 9.4777 - auc: 0.9153 - accuracy: 0.9153
Epoch 16/25
533/533 [==============================] - 153s 285ms/step - loss: 6.9022 - auc: 0.9325 - accuracy: 0.9289
Epoch 17/25
533/533 [==============================] - 154s 286ms/step - loss: 6.3071 - auc: 0.9396 - accuracy: 0.9332
Epoch 18/25
533/533 [==============================] - 154s 286ms/step - loss: 5.2822 - auc: 0.9488 - accuracy: 0.9396
Epoch 19/25
533/533 [========================

In [17]:
!cp ./logs/checkpoints/RGB-model-10.h5 ./drive/MyDrive/checkpoints/rgb-model-10.h5
!cp ./logs/checkpoints/RGB-model-15.h5 ./drive/MyDrive/checkpoints/rgb-model-15.h5
!cp ./logs/checkpoints/RGB-model-20.h5 ./drive/MyDrive/checkpoints/rgb-model-20.h5

In [15]:
history = rgb_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=30,
                        initial_epoch=20,
                        class_weight=class_weight)

Epoch 21/30
533/533 [==============================] - 151s 281ms/step - loss: 3.8759 - auc: 0.9586 - accuracy: 0.9487
Epoch 22/30
533/533 [==============================] - 152s 281ms/step - loss: 2.9074 - auc: 0.9710 - accuracy: 0.9585
Epoch 23/30
533/533 [==============================] - 151s 280ms/step - loss: 2.6168 - auc: 0.9739 - accuracy: 0.9597
Epoch 24/30
533/533 [==============================] - 151s 280ms/step - loss: 2.3565 - auc: 0.9767 - accuracy: 0.9615
Epoch 25/30
533/533 [==============================] - 150s 280ms/step - loss: 2.1508 - auc: 0.9793 - accuracy: 0.9649
Epoch 26/30
533/533 [==============================] - 150s 280ms/step - loss: 1.8371 - auc: 0.9838 - accuracy: 0.9692
Epoch 27/30
533/533 [==============================] - 150s 280ms/step - loss: 1.9915 - auc: 0.9823 - accuracy: 0.9661
Epoch 28/30
533/533 [==============================] - 151s 280ms/step - loss: 1.5529 - auc: 0.9845 - accuracy: 0.9734
Epoch 29/30
533/533 [===========================

In [16]:
!cp ./logs/checkpoints/RGB-model-25.h5 ./drive/MyDrive/checkpoints/rgb-model-25.h5
!cp ./logs/checkpoints/RGB-model-30.h5 ./drive/MyDrive/checkpoints/rgb-model-30.h5

In [18]:
history = rgb_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=35,
                        initial_epoch=30,
                        class_weight=class_weight)

Epoch 31/35
533/533 [==============================] - 151s 281ms/step - loss: 1.0675 - auc: 0.9903 - accuracy: 0.9807
Epoch 32/35
533/533 [==============================] - 150s 279ms/step - loss: 1.0857 - auc: 0.9901 - accuracy: 0.9802
Epoch 33/35
533/533 [==============================] - 150s 280ms/step - loss: 0.9595 - auc: 0.9909 - accuracy: 0.9813
Epoch 34/35
533/533 [==============================] - 151s 281ms/step - loss: 0.8930 - auc: 0.9919 - accuracy: 0.9819
Epoch 35/35
533/533 [==============================] - 151s 280ms/step - loss: 0.8100 - auc: 0.9929 - accuracy: 0.9838


In [19]:
!cp ./logs/checkpoints/RGB-model-35.h5 ./drive/MyDrive/checkpoints/rgb-model-35.h5